In [1]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_trf
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

import re
nlp = en_core_web_trf.load()
# nlp = spacy.load("./custom-ner-with-spacy3/model-best_new")

c:\ProgramData\Anaconda3\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'en_core_web_trf' (3.4.1) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\ProgramData\Anaconda3\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'en_core_web_trf' (3.4.1) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
# opening the file
file_obj = open("./1_1000.txt", "r",encoding="utf-8")
  
# reading the data from the file
file_data = file_obj.read()
  
# splitting the file data into lines
lines = file_data.splitlines()
# print(lines)
file_obj.close()

In [ ]:
for i in range(len(lines)):
    lines[i] = lines[i][20:]

In [ ]:
len(lines)

54663

In [ ]:
for i in lines[:]:
    if 'breaking' in i:
        lines.remove(i)

In [ ]:
len(lines)

36019

In [ ]:
lines[0]

' Why do republicans bear all the burden of cleaning up Joe Biden a mess. Stay woke my friend🤦🏼\u200d♀️ BREAKING: Here is the Republican plan to fight inflation.  1. 2. 3 4. 5.  Any questions?'

In [ ]:
def index_2d(myList, v):
    for i, x in enumerate(myList):
        if v in x:
            return (i, x.index(v))

In [ ]:
def deEmojify(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        u"\u20AC"
        u"\u23F3"
        u"\u2066"
        u"\u2069"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'',text)

In [ ]:
def get_1_mention(text):
    a = 0
    B = []
    words_list = text.split()
    for i in range(len(words_list)):
        if(words_list[i][0]=='@' and a != 1):
            B.append(words_list[i])
            a = 1
        if(words_list[i][0]!='@'):
            B.append(words_list[i])
            a = 0
    return ' '.join(B)

In [ ]:
def get_1_hashtag(text):
    a = 0
    B = []
    words_list = text.split()
    for i in range(len(words_list)):
        if(words_list[i][0]=='#' and a != 1):
            B.append(words_list[i])
            a = 1
        if(words_list[i][0]!='#'):
            B.append(words_list[i])
            a = 0
    return ' '.join(B)

In [ ]:
for i in range(len(lines)):
    tw = re.sub(r"https\S+", '', lines[i])
    tw = re.sub("\n", ' ', tw)
    # tw = re.sub("#", '', tw)
    # tw = re.sub("@[A-Za-z0-9_]+","", tw)
    tw = deEmojify(tw)
    tw = get_1_hashtag(tw)
    tw = get_1_mention(tw)
    tw = re.sub("@", '', tw)
    tw = tw.replace('[','')
    tw = tw.replace(']','')
       
    lines[i] = tw
# lines['text'] = lines
lines[0]

'Why do republicans bear all the burden of cleaning up Joe Biden a mess. Stay woke my friend BREAKING: Here is the Republican plan to fight inflation. 1. 2. 3 4. 5. Any questions?'

In [ ]:
for i in range(len(lines)):
    # lines[i] = re.sub("(?<=[a-zA-Z])'s", " 's", lines[i])
    # lines[i] = re.sub("(?<=[a-zA-Z])\.", " .", lines[i])
    lines[i] = re.sub("Breaking", " ", lines[i])
    lines[i] = re.sub("BREAKING", " ", lines[i])

In [ ]:
# lines = pd.read_csv('./zip/Breaking_3444.csv')['text'].to_list()
list_of_string = []
for i in range(len(lines)):
    list_of_word = [(x.orth_) for x in [y for y in nlp(lines[i])]]
    list_of_string.append(" ".join(list_of_word))
lines = list_of_string

In [ ]:
print(len(lines))

36019


In [ ]:
with open(r'./36k_pre.txt', 'w',encoding="utf-8") as fp:
    for item in lines:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [ ]:
import spacy

doc = nlp(lines[0])
print(doc[0])

displacy.render(doc, style="ent")
entity = []
for ent in doc.ents:
    if ent.label_ == 'PERSON' or ent.label_ == 'GPE' or ent.label_ == 'LOC' or ent.label_ == 'TIME' or ent.label_ == 'DATE':
        entity.append([ent.text, ent.start_char, ent.end_char, ent.label_])
print(entity)
t = lines[1][:entity[0][2]] + '[/' + entity[0][3] + ']' + lines[1][entity[0][2]:]
t = t[:entity[0][1]] + '[' + entity[0][3] + ']' + t[entity[0][1]:]
print(t)


Why


[['Joe Biden', 54, 63, 'PERSON']]
  : House Speaker Nancy Pelosi 's house was broken int[PERSON]o , and h[/PERSON]er husband was violently assaulted .


In [ ]:
def training_json(text):
    text = nlp(text)
    entity = []
    for ent in text.ents:
        if (ent.label_ == 'PERSON' or ent.label_ == 'GPE' or ent.label_ == 'LOC' or ent.label_ == 'TIME' or 
            ent.label_ == 'DATE' or ent.label_ == 'ORG' or ent.label_ == 'NORP' or ent.label_ == 'FAC' ):
            entity.append([ent.text, ent.start_char, ent.end_char, ent.label_])

    for token in text:
        if token.pos_ == 'VERB' and len(token.text) != 1:
            entity.append([token.text, token.idx,token.idx+len(token.text),'ACT'])

    tree_pos = [[x.text,x.idx,x.idx+len(x.text),x.dep_] for x in [y for y in text]]
    place_who = []

    # print(tree_pos)
    for i in range(len(entity)):
        # print(i,entity[i][0])
        # print(index_2d(tree_pos,entity[i][0]))
        try:
            index = index_2d(tree_pos,entity[i][0])[0]
        except:
            continue
        if(entity[i][3] == 'GPE' and 'pass' not in tree_pos[index][3] and 'sub' in tree_pos[index][3]):
            place_who.append([entity[i][0],entity[i][1],entity[i][2],'GPE_WHO'])
        # if(entity[i][3] == 'LOC' and 'pass' not in tree_pos[index][3] and 'sub' in tree_pos[index][3]):
        #     place_who.append([entity[i][0],entity[i][1],entity[i][2],'LOC_WHO'])
        # if(entity[i][3] == 'FAC' and 'pass' not in tree_pos[index][3] and 'sub' in tree_pos[index][3]):
        #     place_who.append([entity[i][0],entity[i][1],entity[i][2],'FAC_WHO'])
    # print(place_who)
    # print(entity)
    for i in range(len(place_who)):
        # print(index_2d(place_who,entity[i][1]))
        index = index_2d(entity,place_who[i][1])[0]
        # print(entity,index)
        # print(index)
        entity.remove(entity[index])
    
    entity.extend(place_who)

    entity = sorted(entity,key=lambda l:l[1])

    t = str(text)
    annotation = [t]
    # print(annotation)

    entities = []
    for i in range(len(entity)):
        if entity[i][0] in entity[i-1][0] and len(entity) != 1:
            continue
        entities.append([entity[i][1],entity[i][2],entity[i][3]])
    # print(entities)
    return entities

training_json('Russia attack Ukraine')


[[0, 6, 'GPE_WHO'], [7, 13, 'ACT'], [14, 21, 'GPE']]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(lines, test_size=0.2,shuffle = False, stratify = None,random_state=69)

In [ ]:
annotation = []

for i in range(len(X_train)):
    training_data = {"classes": [
                            "PERSON",
                            "GPE",
                            "LOC",
                            "TIME",
                            "DATE",
                            "ORG",
                            "ACT","NORP","FAC","EVENT","WORK_OF_ART","PRODUCT","GPE_WHO","LOC_WHO","FAC_WHO"],
                "annotations":[]}
                # ,"GPE_WHO","LOC_WHO","FAC_WHO"
    s = X_train[i]
    ent_json = training_json(X_train[i])
    an = []
    if len(ent_json) != 0:
        an.append(s)
        an.append({'entities':ent_json})
        annotation.append(an)
        # print(annotation)
    training_data['annotations'] = annotation
#     print(annotation)
# print(annotation)

In [ ]:
training_data['annotations'][599]

["  : Jerry Lee Lewis , ' Great Balls of Fire ' singer , dead at 87",
 {'entities': [[4, 19, 'PERSON'], [63, 65, 'DATE']]}]

In [ ]:
import json
with open(r'./custom-ner-with-spacy3/0.8_36k.json', 'w',encoding="utf-8") as fp:
    fp.write(json.dumps(training_data))
    print('Done')

Done


In [ ]:
annotation = []
for i in range(len(X_test)):
    training_data = {"classes": [
                            "PERSON",
                            "GPE",
                            "LOC",
                            "TIME",
                            "DATE",
                            "ORG",
                            "ACT","NORP","FAC","EVENT","WORK_OF_ART","PRODUCT","GPE_WHO","LOC_WHO","FAC_WHO"],
                "annotations":[]}
                # ,"GPE_WHO","LOC_WHO","FAC_WHO"
    s = X_test[i]
    ent_json = training_json(X_test[i])
    an = []
    if len(ent_json) != 0:
        an.append(s)
        an.append({'entities':ent_json})
        annotation.append(an)
        # print(annotation)
    training_data['annotations'] = annotation
#     print(annotation)
# print(annotation)

In [ ]:
import json
with open(r'./custom-ner-with-spacy3/0.2_36k.json', 'w',encoding="utf-8") as fp:
    fp.write(json.dumps(training_data))
    print('Done')

Done


In [ ]:
len(X_train)

28815

In [ ]:
len(X_test)

7204

In [ ]:
X_train

['Why do republicans bear all the burden of cleaning up Joe Biden a mess . Stay woke my friend   : Here is the Republican plan to fight inflation . 1 . 2 . 3 4 . 5 . Any questions ?',
 "  : House Speaker Nancy Pelosi 's house was broken into , and her husband was violently assaulted .",
 '  : The break in at Nancy Pelosi ’s house is suspected to be targeted , law enforcement sources tell ABC News . The suspect allegedly entered the house through a sliding glass door , carrying a hammer , and was apparently looking for the House Speaker herself . 1/2',
 '  : Republican Gov. Glenn Youngkin disgustingly responds to the hammer attack on Paul Pelosi in his home by saying that he \'s " gon na send " Nancy Pelosi " back to be with him in California . " RT IF YOU STAND WITH THE PELOSIS !',
 '  : Paul Pelosi is undergoing brain surgery . There is NO place for violence in our democracy . This horrible attack MUST be condemned by Marjorie Taylor Greene , Tucker Carlson , and Donald Trump NOW !',
